# Model Project: The Agents Labour Supply Decision


In present project we seek to model the supply of labour by studying a two-good static problem, where agents value leisure and consumption. As an extension for the base model we impose a progressive tax system, which creates kinks in the budget constraint. We then show, how this may create bunching around tax bracket cut-offs.

The basic problem of the agents is:


$$\begin{aligned}  \max_{c,l}\ \ &u(c,l) \quad s.t. \ \ R_0 = wl + c  \quad c,l\geq0\\
 &\dfrac{\partial u}{\partial a}= u_a>0 \quad  \wedge \quad \dfrac{\partial^2 u}{\partial a^2}=u_{aa}<0,  \qquad a=c,l \\
 &R_0=wT + A_0 
 \end{aligned} $$
 

Where $R_0$ is the income potential of the agent, $c$ the consumption which is a numeraire, $l$ is the amount of leisure consumed with the associated oppurtunity cost of being the wage rate, $w$. We assume marginal utility of consumption and leisure are positive but decreasing. Further we for now assume the income potential is the sum of wage earnings from working your entire time endowment, $T$, and non-labour income, $A_0$. The problem is framed such that the agent at default suplies all his time as labour, giving her/him an income of $R_0$. The agent now has to decide how much leisure she/he will 'buy back', considering the consumption potential which is implied.

The Langrangian for the problem is:
$$ \mathcal{L}(c,l; \lambda)= u(c,l)\ + \ \lambda( R_0 - wl - c) $$ 
For which the first order conditions with regards to $c$ and $l$ implies an optimum at $\dfrac{u_l}{u_c}=w$ as the constraint is binding.

## Numerical optimization

We will solve this problem using a numerical optimizer, which will allow for easy handling of extensions to this basic model. 

Dependencies:

In [80]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy.optimize import Bounds
import scipy.integrate as integrate
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

First we code up two utility functions, a CES and a Cobb-Douglas, but any utility function that observes the above restrictions can be implemented.

In [81]:
# Cobb-Douglas:
def  _cobbdouglas(c,l, alpha):
    """ Cobb douglas utility function for 2 goods.
    PARAMETERS
    alpha: relative preference for consumption and leisure
    INPUT:
    c: consumption
    l: leisure

    OUTPUT:
    u: utility (negative)"""
    u= (c**alpha)*l**(1-alpha)
    return -u
def  cobbdouglas(x,alpha):
    return _cobbdouglas(c=x[0],l=x[1], alpha=alpha)

# CES utility function:
def _CES(c,l):
    """ CES utility function for 2 goods.
    PARAMETERS:
    r: 1/(1-r) is the elasticity of substitution, r<=1.
    a: relative preference for consumption (1 unit of consumption
     gives the same utility as 1 unit lesiure), 0<=a<=1.
    INPUT:
    c: consumption (dollars spent)
    l: leisure (hrs pr. week)

    OUTPUT:
    u: utility (neg) """
    u=(a*c**r + (1-a)*l**r)**(1/r)
    return -u
def CES(x):
    """ Takes a tuple as argument:
    ARGS
    x is a 2x1
    x0= consumption
    x1= leisure
    OUTPUT
    utility from consumption
    """
    return _CES(c=x[0],l=x[1]) 

We now have to define the constraints. Three moving parts are specified for the budget constraint (BC):<br>
1) Wage function: returning the after tax wagerate for for a given level of leisure consumption (the more leisure the lower taxrate = higher wagerate).<br>
2) Leisure expenditure function: returns the expenditure incurred from buying 'l' units of leisure. <br>
3) The budget function: calculates the slack in budget given choices of leisure and consumption (the actual constraint). <br>

In [82]:
# 1) Wage function: returning the after tax wagerate for for a given level of leisure consumption (the more leisure the lower taxrate = higher wagerate).<br>
def wage_flat(l):
    """ Marg. wage function with no taxation:
        
        PARAMETERS:
        w: constant wage
        ARGS:
        l: leisure 
        
        OUTPUT:
        The marginal after tax wage rate
    """
    return w
#2) Leisure expenditure function:
def leiexp(l,wage):
    """ Calculates the expenditure from buying l units of leisure given a wage function as the definite integral
    from 0 to l of the wage function.
    ARGS:
    l: leisure consumed (int)
    Wage: Marginal wage-function (incoperating tax system) (function)
    
    OUTPUT:
    leiexp: expenditure on leisure (int)
    """
    return sp.integrate.quad(wage,0,l)[0]

#3) Budget constraint
def budget(x, wage, maxlabinc):
    """The budget functin calculates the slack for consumption plan:
    ARGS
    x (2-tuple)
         x[0]: consumption of good
         x[1]: consumption of leisure
    wage(l)  : marg. wage/price of leisure (function)
    maxlabinc: Labour income from working T units of time (int)
    
    OUTPUT:
    slack: 'Leftover' given consumption plan x.
    """
    c=x[0]
    l=x[1]
    R0= maxlabinc + A
    slack= R0 - leiexp(l, wage) - c
    return  slack
def budget_func(wage,maxlabinc):
    """ Produces the budget constraint to input in maximizer.  It is a equality contraint -> slack shall be zero in optimum
    as marg. utility of both goods is stricty positive.
    ARGS:
    Wage: marg. wage function
    maxlabinc: integral of wage fucntion from 0 to T
    OUTPUT:
    Budget Contraint: Dict. specifying the constraint to be used in scipy.optimize.minimize
    """
    return {'type':'eq', 'fun':budget, 'args': [wage, maxlabinc]} 

Note: 1) that we define the wage as a function this allows us to extend the model, 2) we have yet to define the bounds on consumption/leisure, as these depend on T, we will define them within when parameters are chosen.



## Example: Cobb Douglas utility
For a Cobb-Douglas utility functions, $u(c,l)= c^\alpha l^{1-\alpha}$, the solution Marshallian demand functions are:
$$c^*= w l \left(\dfrac{\alpha}{1-\alpha}\right)=R_0 \alpha \qquad l^*=\dfrac{R_0 (1-\alpha)}{w}$$
These can be obtained by isolating factor demand for one good from the optimum condition and inserting this into the budget constraint.
However to initially test our program, we will compare the analytical solution for a particular set of parameters to the optimizer:

In [83]:
# Budget related parameters
T=10 # 10 units of time endowed
w=10 # flat hourly wage of 10
A=20 # 20 income from non labour sources
maxlabinc=leiexp(T,wage_flat)
# Utility related parameters
alpha=0.5 # Assuming

# Optimizer setup:
bounds= Bounds([0,np.inf], {0, T})

Given these parameters, we find $R_0=10\times10 + 20$, and analytical solution for optimal consumption choices are $c^*=120\times0.5=60$ and $l^*=\dfrac{120\times0.5}{10}=6$. <br>
The numerical optmization is performed by the 'SLSQP' algorithm for constrained optimization included in sci.py:

In [84]:
guess=(5, 70) 
result = optimize.minimize(cobbdouglas,guess,args=alpha,
                             method='SLSQP',
                             constraints=[budget_func(wage,maxlabinc)],
                             options={'disp':True})
 

print('\nx = ',result.x)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -18.97366574611347
            Iterations: 10
            Function evaluations: 40
            Gradient evaluations: 10

x =  [59.99096963  6.00090304]


Deviations in the analytical and numerical solutions can be chalked up to numerical precision.

## Progressive tax system

First we code up a wage function, which accounts for changes in marginal taxation:

In [75]:
#1) Wage function 
def wage_prog(l):
    """ Wage function for a progressive tax system with allowing for two specified kinks
        2 kinks (3 tax brackets).
        
        PARAMETERS:
        cut1, cut2: cutoff for tax brackets defined by consumption of
                    leisure.
        w0,w1,w2: wage rates in the respective tax brackets.
        
        ARGS:
        demand typle, x: x[0] cons, x[1] lesiure consumption
        
        OUTPUT:
        w0,w1,w2: the appropriate marginal after tax wage rate
    """
    if l < l_top:
        return w*(1-toptax)
    elif l_top<= l and l < l_bot:
        return w*(1-bottax)
    else:
        return w

The tax system has 3 brackets: no tax, bottom tax and top tax. Due to the way we frame the problem, this amounts to the marginal cost of leisure increasing in the demand for leisure. This is to say, the more leisure an agent consumes the lower his marginal tax is (because earnings are low), thus the marginal price (after tax wage rate) of leisure increases the more leisure is consumed. This can be expressed as:
$$w(l) = \begin{cases} & w \quad &\text{for } l> l_{bottom} \\
                       & w \times (1-t_{bottom}) \quad &\text{for } l_{top} <l < l_{bottom}  \\
                       & w \times (1-t_{top}) \quad &\text{for } l < l_{top}  \end{cases}, \quad l_{bottom}> l_{top} \ \wedge \ t_{bottom}<t_{top}$$

We implement a tax system, where the first 1/3 of the time endowment supplied as labour is not taxed, the next 1/3 of time endowment is taxed in the bottom tax bracket, and additional labour is taxed in the top tax bracket. The marginal tax in the bottom tax bracket is set to 30% and in the top tax bracket to 60%. When framed as buying leisure this implies:

$$w(l) = \begin{cases} & w \quad &\text{for } l> T\times \dfrac{2}{3} \\
                       & w \times (1-0.3) \quad &\text{for } T \times \dfrac{2}{3} >l > T\times \dfrac{1}{3}  \\
                       & w \times (1-0.6) \quad &\text{for } T\times \dfrac{1}{3}>l  \end{cases}$$

Now we simulate agents with differing draws of $\alpha$, where $\alpha_i \sim$ uniform$[0,1]$, and use numerical optmization to find their optimal consumption of leisure and goods:

In [95]:
# Set wage function
l_top= 1/3*T # Work more than 2/3 of your time and pay top tax
l_bot= 2/3*T # work more than 1/3 of your time and pay bottom tax
toptax= 0.6
bottax= 0.3
np.random.seed(2019); # set the seed
n = 10; # number of agents simulated

# Draw agents
alpha = np.random.uniform(0,1,size=n)

# Run optimizer

def solver(alpha):
    """ Runs optimizer over a tuple of parameterizations (alpha's).
    """
    result = optimize.minimize(cobbdouglas,guess,args=alpha,
                             method='SLSQP',
                             constraints=[budget_func(wage,maxlabinc)],
                             options={'disp':False})
    return (result.x[1]
results=[solver(x) for x in alpha]

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [91]:
print(results)
print(alpha)

[47.16562821  7.28343718]
[0.90348221 0.39308051 0.62396996 0.6378774  0.88049907 0.29917202
 0.70219827 0.90320616 0.88138193 0.4057498 ]
